# Introduction

This Notebook is a step for step handbook for getting per Unit Generation Data out of the Smard Data. It isn't necessary to run this notebook, the processed data can also be downloaded directly from the repository. This is just for understanding how the data was processed.

As Data Source the Smard Powerplant Data was used and processed so it can be handled with the EIC IDs of each Unit.


 

## Preparation of Python environment

The following modules and their dependencies are required to run this notebook:
* Pandas (with openpyxl)
* Numpy
* datetime 

In [28]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import read_excel
import os
import datetime


## Download of input files and setup of folder structure

First the raw primary Data was downloaded and prepared. Two different Datasets were used

- First the _blocks.xlsx_ data, which is a list of all powerplants in Germany. It was collected by the INATECH and is available via the repository.

- For the generation data csv-files were downloaded from the Smard Website:
https://www.smard.de/en/downloadcenter/download-power-plant-data/?downloadAttributes=%7B%22selectedPowerPlant%22:%22all%22,%22selectedContent%22:%22generation%22,%22from%22:1451602800000,%22to%22:1483225199999,%22selectedFileType%22:%22CSV%22%7D
It was downloaded for each year independently from year 2016 to 2021. For that following options were selected:
    - _All Power Plants_
    - _Content: Actual generation_
    - _01.01.Year - 31.12.Year_ where you choose for year each year from 2016-2021
    - _Select resolution: original resolution_
    - _CSV_

 Then each csv file was put into the _smard_ folder (without a Subfolder). So in the end all csv files are in the __smard__ folder.

In [29]:
# INITIAL SETUP

# data is currently provided for the years of 2016 to 2022
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022]

# import main file for German powerplant data including EIC, ETS-ID, electrical and heat power, CHP
blocks = read_excel('input/blocks.xlsx')

blocks.head(10)

,ETS-ID,EIC,Name BNA,City,Plant name,Block name,Filename,Column_Block_Smard,Column_Block_Smard_2
0,1399,11WD7BERG1S--A-X,Bergkamen A,Bergkamen,Bergkamen,A,Bergkamen,Generation_DE Bergkamen A [MW],NaN
1,1453,11WD8BOXB1L---N8,Boxberg Block N,Boxberg,Boxberg,N,Boxberg,Generation_DE Boxberg Block N [MW],NaN
2,1453,11WD8BOXB1L---P4,Boxberg Block P,Boxberg,Boxberg,P,Boxberg,Generation_DE Boxberg Block P [MW],NaN
3,1454,11WD8BOXB1L---Q2,Boxberg Block Q,Boxberg,Boxberg,Q,Boxberg,Generation_DE Boxberg Block Q [MW],NaN
4,1454,11WD8BOXB1L---R0,Boxberg Block R,Boxberg,Boxberg,R,Boxberg,Generation_DE Boxberg Block R [MW],NaN
5,1460,11WD8LIPD1L---R6,Kraftwerk Lippendorf Block R,Boehlen,Lippendorf,R,Braunkohlekraftwerk_Lippendorf,Generation_DE Kraftwerk Lippendorf Block R [MW],NaN
6,1460,11WD8LIPD1L---S4,Kraftwerk Lippendorf Block S,Boehlen,Lippendorf,S,Braunkohlekraftwerk_Lippendorf,Generation_DE Kraftwerk Lippendorf Block S [MW],NaN
7,795,11WD2BURG000145R,Gasturbinenanlage 12,Burghausen,Burghausen,NaN,Burghausen_GT,Generation_DE Gasturbinenanlage 12 [MW],NaN
8,1419,11WD2BUSD0000386,Buschhaus,Helmstedt,Buschhaus,NaN,Buschhaus,Generation_DE D [MW],NaN
9,1409,11WD7HERD2G-H6-X,Cuno Heizkraftwerk Herdecke H6,Herdecke,Cuno HKW Herdecke,H6,Cuno_Heizkraftwerk_Herdecke,Generation_DE Cuno Heizkraftwerk Herdecke H6 [MW],NaN


In [30]:
# GENERATION OF EIC AND ETS-ID LISTS

# list of all powerplant block EIC codes
eic_list = blocks['EIC'].unique()

# list of all powerplant location ETS-ID codes
ets_list = blocks['ETS-ID'].unique()

### Prepare the smard data
In the smard data we have a csv-file for each powerplant and year. We generate a Dataframe which maps 
the files to the correct plant name. This plant name is also present in the column _filename_ in the _blocks.xlsx_ data.

With this way we can map the blocks represented in the _blocks.xlsx_ to the correct csv file

In [31]:
# GENERATE MAPPING FROM CSV FILE TO BLOCK

# Folder which contains all smard data files
smard_folder = "./input/smard"
files_smard = os.listdir(smard_folder)

# Initialize columns for DataFrame, in which we save the year, the plant name and the filename
years_smard = []
plant_names_smard = []
filenames_smard = []

# Loop through all filenames
for filename in files_smard:
    # Split the filename on underscore, to get each information saved in the filename
    filename_array = filename.split("_")
    
    # Check if file is csv file
    if filename_array[-1].split(".")[1] != "csv":
        continue
    
    # Get plant name out of the filename
    plant_name = "_".join(filename_array[:-4])
    
    # Check if Plant name is in the blocks.xlsx file, if not continue
    if  plant_name not in blocks["Filename"].values:
        continue

    # Save variables in the corresponding array
    filenames_smard.append(filename)
    years_smard.append(filename_array[-3][:4])
    plant_names_smard.append(plant_name)

# Convert everything to a DataFrame
smard_files_df = pd.DataFrame({"year":years_smard, "plant name":plant_names_smard, "filename":filenames_smard}) 

smard_files_df.head(10)

,year,plant name,filename
0,2016,Bergkamen,Bergkamen_201601010000_201612312359_hour_6.csv
1,2017,Bergkamen,Bergkamen_201701010000_201712312359_stunde_6.csv
2,2018,Bergkamen,Bergkamen_201801010000_201812312359_stunde_6.csv
3,2019,Bergkamen,Bergkamen_201901010000_201912312359_stunde_6.csv
4,2020,Bergkamen,Bergkamen_202001010000_202012312359_stunde_6.csv
5,2021,Bergkamen,Bergkamen_202101010000_202112312359_hour_6.csv
6,2022,Bergkamen,Bergkamen_202201010000_202212312359_stunde_6.csv
7,2016,Boxberg,Boxberg_201601010000_201612312359_hour_8.csv
8,2017,Boxberg,Boxberg_201701010000_201712312359_stunde_8.csv
9,2018,Boxberg,Boxberg_201801010000_201812312359_stunde_8.csv


### Define helper functions for processing of Smard Data

In the following we define function we use for the processing:
1. __rename_column__: This function is used in the pd.DataFrame.apply function and get's rid of the Ending at each column, because it differs from year to year (Originalauflösung or original resolution) 

1. __rename_columns__: This function is used to rename the columns in the csv files, so they match with the column names saved in the smard_files_df. Therefore we first iterate over each column and rename it with the function described above

1. __to_datetime__: This function converts the Time Format given in the SMARD Data to a Datetime Format

1. __process_dataframe__: This function processes the Data for one Generation unit, where it does the following: 
    - Change Datatype of Dataframe to numeric
    - use to_datetime function to get correct timestamps 
    - renames the column from the _eic-id_ to _Generation [MW]_ 
    - fills up missing timestamps with NaN so every EIC-ID has 8760 (8784) rows



In [32]:
def rename_column(column_name):
    # Split the name by " "
    column_name_split = column_name.split(" ")

    # Get the index of "[MW]", as this is in each column of every file
    index_mw = column_name_split.index("[MW]")

    # join again but only including until "[MW]" 
    column_name_new = " ".join(column_name_split[:index_mw+1])
    
    return column_name_new

In [33]:
def rename_columns(smard_data_df):
    
    # Rename Columns so they can be matched to the entries in the Blocks Dataset
    smard_data_df = smard_data_df.rename(lambda x: rename_column(x) if "[MW]" in x else x, axis=1)

    # Rename First 3 Columns so all are named the same
    new_column_names = ["Date", "Start", "End"]
    for i in range(3): smard_data_df.columns.values[i] = new_column_names[i]

    return smard_data_df


In [34]:
# Function to convert the Time Format given in the SMARD Data to a Datetime Format
def to_datetime(x):
    
    # Extract Date from Dataframe
    date = str(x["date"])
    date_len = len(date)
    if  date_len == 8 or date_len == 7:
        year = int(date[-4:])
        month = int(date[-6:-4])
        day = int(date[:-6])
    else:
        print(date)
        return "test"
        
    # Get hour from x
    hour = int(x[1].split(":")[0])
    x_date = datetime.datetime(year=year, month=month, day=day, hour=hour)
    return x_date

In [35]:
# Function to process the Data for one Generation unit
def process_dataframe(smard_data_df, eic_id, year):

    # Change Datatype of column to be able to handle data as numbers
    smard_data_df[eic_id] = pd.to_numeric(smard_data_df[eic_id], errors="coerce")

    # Add Column Timestamp, which hase the Starting hour of the slot as a Datetime Format
    smard_data_df["Timestamp"] = smard_data_df.apply(lambda x: to_datetime(x), axis=1)

    # Copy the Columns Timestamp, EIC-ID, from the current DataFrame, which will be in the output Dataframe
    smard_data_long_df = smard_data_df[["Timestamp", eic_id]].copy()

    # Rename the Column {eic_id} to Generation, because this contains the Generation Data
    smard_data_long_df = smard_data_long_df.rename(columns={eic_id: "Generation [MW]"})

    # Fill up missing rows with NaNs, so each Generation Unit has a row for each hour
    new_index = pd.Series(pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31 23:00:00', freq='H'))
    smard_data_long_df = smard_data_long_df.drop_duplicates(["Timestamp"], keep="first")
    smard_data_long_df = smard_data_long_df.set_index("Timestamp")
    smard_data_long_df = smard_data_long_df.reindex(new_index, fill_value=np.nan)

    # Add a Column EIC-ID with the eic_id as value
    smard_data_long_df["EIC-ID"] = eic_id
    
    return smard_data_long_df

## Process Data for general use
Now the final processing is done, where a long csv for each year is generated containing all Generation Data of each Unit.
To do that, we loop over all power plants in the smard folder. Then for each file we get the correspoding blocks by mapping the file to the _blocks.xlsx_, as well as the correct filename for power plant and year.

Then the file is loaded and the columns are renamed. 

In [36]:
# Loop over years to get Generation Data for each year
for year in years:

    # Initialize the Dataframe
    generation_data_long_df = pd.DataFrame(data=[], columns=["EIC-ID", "Generation [MW]"])
    
    # Loop over each Power Plant listed as CSV-File, get the smard Data and calculate the yearly Generation Data
    for smard_file in smard_files_df["plant name"].unique():

        # Get all Blocks belonging to one CSV-File
        blocks_file_df = blocks[blocks["Filename"] == smard_file]

        # Check if DataFrame is emtpty
        if len(blocks_file_df) == 0:
            continue

        # Get the Plant Name and corresponding column names
        plant_name = blocks_file_df["Plant name"].values[0]
        column_names = blocks_file_df["Column_Block_Smard"].unique()

        # With the Plant Name get the correct csv-filename for each Year
        filename = smard_files_df[(smard_files_df["plant name"] == smard_file) & (smard_files_df["year"] == str(year))]["filename"].values
        
        # Check if a filename was found, if not throw an Error and continue
        if len(filename) == 0: 
            bna_name = blocks_file_df["Name BNA"].values
            print(f"WARNING: No CSV Filename found for {bna_name} in year {year}")
            continue
        filename = filename[0]

        # Load CSV File
        smard_data_df = read_csv(f"{smard_folder}/{filename}", sep=";", decimal=",", thousands=".", na_values=["-"])

        # Use function above to rename the columns 
        smard_data_df = rename_columns(smard_data_df)

        # Loop over all Blocks belonging to the csv file which are defined in the blocks dataset
        for column_name in column_names:

            # Boolean to handle multiple columns
            multiple_columns = False

            # Get EIC Ids of the Column name
            eic_ids_column = blocks_file_df[blocks_file_df["Column_Block_Smard"] == column_name]["EIC"].values

            # Check if there is just one Columns for multiple EIC-IDS
            # If yes each generation entry in the column will be devided by the amount of different EIC IDS and split into multiple Columns
            amt_eic_ids = len(eic_ids_column)
            if amt_eic_ids > 1:
                multiple_columns = True

            # Use Try and Catch to handle multiple column names, as some Blocks are named different in seperate years
            try:
                # Check if One column represents multiple EIC-IDs
                if multiple_columns:

                    # Create one new column per EIC-ID, where the EIC ID is the name of the column
                    for eic_id in eic_ids_column:
                        
                        # Each column get's the equal share of the Values, 
                        # so they're calculated by dividing the Values in the original column by the amount of EIC-IDs 
                        smard_data_df[eic_id] = smard_data_df[column_name] / len(eic_ids_column)
                        smard_data_long_df = process_dataframe(smard_data_df=smard_data_df, eic_id=eic_id, year=year)

                else:
                    # If it is just one EIC-ID for that column get the EIC-ID out of the eic_ids_column list
                    eic_id = eic_ids_column[0]

                    # Rename the columns so the correct column will be changed to EIC-ID as Column name
                    smard_data_df.columns = [eic_id if x== column_name else x for x in smard_data_df.columns]
                    smard_data_long_df = process_dataframe(smard_data_df=smard_data_df, eic_id=eic_id, year=year)
            
            # Throws error when Column name doesn't exist, then try second column name
            except Exception as error:

                # Try second Column name, if that also not works continue 
                try:
                    # Get second name of Column in csv File for that Block
                    column_name = blocks_file_df[blocks_file_df["EIC"] == eic_id]["Column_Block_Smard_2"].values[0]
        
                    # Rename the columns so the correct column will be changed to EIC-ID as Column name
                    smard_data_df.columns = [eic_id if x == column_name else x for x in smard_data_df.columns]
                    smard_data_long_df = process_dataframe(smard_data_df=smard_data_df, eic_id=eic_id, year=year)
                
                except Exception as error:
                    continue

            # Concat the current Dataframe to the long Dataframe
            generation_data_long_df = pd.concat([generation_data_long_df, smard_data_long_df], ignore_index=False)

    # Save yearly generation as csv-file
    generation_data_long_df.to_csv(f"output/general/generation_data_long_{year}.csv")

generation_data_long_df.head(20)

            

,EIC-ID,Generation [MW]


## Process Data for use with ETS-IDs 

With this step the data is processed so it can be easily used with ETS-IDs. Therefore one csv-file for each ETS-ID and year is generated, with the EIC-IDs as columns. 

So the Matching betweeen EIC-IDs (Generation) and ETS-IDS (Emissions) is already implemented and further data processing ca be done easily.

In [37]:
# IMPORTING THE GENERATION DATA FROM THE SMARD CSV FILES

# The columns we add to the installations_df, for each year a Columns with the Generation per Year
columns = ["EIC"] + [f'Generation elec. {y} [MWh_el]' for y in years]

# Loop over each File in the smard_files Dataframe and calculate the yearly Generation Data
for plant_name in smard_files_df["plant name"].unique():

    # Get all Blocks belonging to one Plant Name
    blocks_smard_df = blocks[blocks["Filename"] == plant_name]

    # Check if DataFrame is emtpty
    if len(blocks_smard_df) == 0:
        continue

    # Get all ETS-IDs, mostly it's just one id but for some Plants there is more than one ID
    ets_ids_blocks = blocks_smard_df["ETS-ID"].unique()

    # Loop over years to get Generation Data for each year
    for year in years:

        # With the Plant Name get the correct csv-filename for ETS-ID and Year
        filename = smard_files_df[(smard_files_df["plant name"] == plant_name) & (smard_files_df["year"] == str(year))]["filename"].values
        
        # Check if a filename was found, if not throw an Error and continue
        if len(filename) == 0: 
            bna_name = blocks_smard_df["Name BNA"].values
            print(f"WARNING: No CSV Filename found for {bna_name} in year {year}")
            continue
        filename = filename[0]

        # Read in CSV Data
        smard_data_df = read_csv(f"{smard_folder}/{filename}", delimiter=";", thousands=".", na_values="-")

        # Use function above to rename the columns 
        smard_data_df = rename_columns(smard_data_df)

        # Loop over ETS-IDs to be able to seperate Generation Data into ETS seperated CSV Files
        for ets_id in ets_ids_blocks:

            # Get the Blocks which have all the same ETS ID out of the Blocks
            # with the same filename
            blocks_ets_df = blocks_smard_df[blocks_smard_df["ETS-ID"] == ets_id]

            # Get the corresponding EIC IDs
            eic_ids = blocks_ets_df["EIC"].values

            # Get columns Names to be able to handle multiple EICs for one column
            column_names = blocks_ets_df["Column_Block_Smard"].unique()

            # Loop over all Blocks belonging to the csv file which are defined in th blocks dataset
            # and have the same ETS ID
            for column_name in column_names:

                # Boolean to handle columns with multiple EIC-IDs
                multiple_columns = False

                # Get EIC Ids of the Column name
                eic_ids_column = blocks_ets_df[blocks_ets_df["Column_Block_Smard"] == column_name]["EIC"].values

                # Check if there is just one Columns for multiple EIC-IDS
                # If yes each generation entry in the column will be devived by the amount of differenc EIC IDS and split into multiple Columns
                amt_eic_ids = len(eic_ids_column)
                if amt_eic_ids > 1:
                    multiple_columns = True

                # Use Try and Catch to handle multiple column names, as some Blocks are named different in seperate years
                try:
                    # Check if One column represents multiple EIC-IDs
                    if multiple_columns:

                        # Change Datatype of column
                        smard_data_df[column_name] = pd.to_numeric(smard_data_df[column_name], errors="coerce")

                        # Create one new column per EIC-ID, where the EIC ID is the name
                        for eic_id in eic_ids_column:
                            
                            # Each column get's the equal share of the Values, 
                            # so they're calculated by dividing the Values in the original column by the amount of EIC-IDs 
                            smard_data_df[eic_id] = smard_data_df[column_name] / len(eic_ids_column)
                    else:
                        # If it is just one EIC-ID for that column get the EIC-ID out of the eic_ids_column list
                        eic_id = eic_ids_column[0]

                        # Change Datatype of column to be able to use the pandas.sum() method
                        smard_data_df[column_name] = pd.to_numeric(smard_data_df[column_name], errors="coerce")

                        # Rename the columns so the correct column will be changed to EIC-ID as Column name
                        smard_data_df.columns = [eic_id if x== column_name else x for x in smard_data_df.columns]
                
                # Throws exception when column_name doesn't exist in thes smard_data_df
                # Then try the second column name, if it exist. When this also fails, continue
                except Exception as error:

                    try:
                        # Get second name of Column in csv File for that Block
                        column_name = blocks_ets_df[blocks_ets_df["EIC"] == eic_id]["Column_Block_Smard_2"].values[0]

                        # Change Datatype of column to be able to use the pandas.sum() method
                        smard_data_df[column_name] = pd.to_numeric(smard_data_df[column_name], errors="coerce")
                        
                        # Rename the columns so the correct column will be changed to EIC-ID as Column name
                        smard_data_df.columns = [eic_id if x == column_name else x for x in smard_data_df.columns]
                    
                    except Exception as error:
                        print(error)
                        continue

            # Get all Columns containing Generation Data but not belonging to the defined Units with EIC-ID
            column_drops = [column for column in smard_data_df.columns.values[3:] if column not in eic_ids]

            # Delete the columns not having the correct EIC-IDs
            # Use copy, because smard_data_df might be used again in the next loop, when multiple ETS-IDs exist
            smard_data_df_output = smard_data_df.drop(column_drops, axis=1).copy()

            # # Rename First 3 Columns so all are named the same
            new_column_names = ["date", "start", "end"]
            for i in range(3): smard_data_df_output.columns.values[i] = new_column_names[i]

            # Get timestamp by date and time, where the timestamop defines the start time of the hourly generation data
            smard_data_df_output["Timestamp"] = smard_data_df_output.apply(lambda x: to_datetime(x), axis=1)

            # Set timestamp as index
            smard_data_df_output = smard_data_df_output.set_index(["Timestamp"])

            # Drop the columns ["date", "start", "end"], because we just need the timestamp
            smard_data_df_output = smard_data_df_output.drop(new_column_names, axis=1)

            # Save as CSV file with the name pattern {plant_name}_{ets_id}_{year}.csv
            smard_data_df_output.to_csv(f"output/ets/{plant_name}_{ets_id}_{year}.csv")

No data for submitted query
Keine Daten für gegebene Anfrage
No data for submitted query
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
No data for submitted query
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
No data for submitted query
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
No data for submitted query
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
Keine Daten für gegebene Anfrage
No data for submitted query
Keine Daten für gegebene Anfrage
 [MW] Original resolutions
No data for submitted query
 [MW] Original resolutions
No data for submitted query
 [MW] Originalauflösungen
Keine Daten für gegebene Anfrage
 [MW] Originalauflösungen
Keine Date